In [66]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-12-20 00:46:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-12-20 00:46:47--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-12-20 00:46:47--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [71]:
!unzip glove*.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: glove.6B.50d.txt        
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.6B.100d.txt       
replace glove.6B.200d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.6B.200d.txt       
replace glove.6B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.6B.300d.txt       


In [73]:
!ls
!pwd

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip    index.html
glove.6B.200d.txt  glove.6B.50d.txt   glove.6B.zip.1  sample_data
/content


In [102]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [103]:
print('Loading word vectors...')
word2vec = {}
embedding = []
idx2word = []
with open('glove.6B.100d.txt') as f:
    for row in f:
        value = row.split(" ")
        word = value[0]
        vec = np.asarray(value[1:], dtype = 'float32')
        word2vec[word] = vec
        embedding.append(vec)
        idx2word.append(word)

print('Found %s word vectors.' % len(word2vec))
embedding = np.array(embedding)
V,D = embedding.shape
print(embedding.shape)

Loading word vectors...
Found 400000 word vectors.
(400000, 100)


In [104]:
def find_analogies(w1, w2, w3):
    for w in (w1, w2, w3):
        if w not in word2vec:
            print("%s not in dictionary" % w)
            return
    w1vec = word2vec[w1]
    w2vec = word2vec[w2]
    w3vec = word2vec[w3]
    w4vec = w1vec + w3vec - w2vec

    distance = pairwise_distances(w4vec.reshape(1,D), embedding, metric = 'l2').reshape(V)
    ids = distance.argsort()[:4]
    words = [idx2word[id] for id in ids if idx2word[id] not in (w1, w2, w3)]
    print('\n',w1, "-", w2, "=", words[0], "-", w3)

In [105]:
find_analogies('king', 'man', 'woman')


 king - man = queen - woman


In [106]:
find_analogies('france', 'paris', 'london')


 france - paris = britain - london


In [107]:
find_analogies('france', 'paris', 'rome')


 france - paris = italy - rome


In [108]:
find_analogies('paris', 'france', 'italy')


 paris - france = rome - italy


In [109]:
find_analogies('france', 'french', 'english')


 france - french = england - english


In [110]:
find_analogies('japan', 'japanese', 'chinese')


 japan - japanese = china - chinese


In [111]:
find_analogies('japan', 'japanese', 'italian')


 japan - japanese = italy - italian


In [112]:
find_analogies('december', 'november', 'june')


 december - november = july - june


In [113]:
find_analogies('miami', 'florida', 'texas')


 miami - florida = dallas - texas


In [114]:
find_analogies('einstein', 'scientist', 'painter')


 einstein - scientist = picasso - painter


In [115]:
find_analogies('man', 'woman', 'she')


 man - woman = he - she


In [116]:
find_analogies('man', 'woman', 'aunt')


 man - woman = uncle - aunt


In [117]:
find_analogies('man', 'woman', 'sister')


 man - woman = brother - sister


In [118]:
find_analogies('man', 'woman', 'wife')


 man - woman = brother - wife


In [119]:
find_analogies('man', 'woman', 'actress')


 man - woman = actor - actress


In [120]:
find_analogies('man', 'woman', 'mother')


 man - woman = father - mother


In [121]:
find_analogies('nephew', 'niece', 'aunt')


 nephew - niece = uncle - aunt


In [122]:
def nearest_neighbors(w1):
  if w1 not in word2vec:
    print("%s not in dictionary" % w1)
    return
  w1vec = word2vec[w1]
  distance = pairwise_distances(w1vec.reshape(1,D), embedding, metric = 'l2').reshape(V)
  ids = distance.argsort()[:11]
  words = [idx2word[id] for id in ids if idx2word[id]!=w1]
  print(words)

In [123]:
nearest_neighbors('king')

['prince', 'queen', 'monarch', 'brother', 'uncle', 'nephew', 'son', 'throne', 'kingdom', 'reign']


In [124]:
nearest_neighbors('france')

['belgium', 'britain', 'french', 'spain', 'paris', 'europe', 'netherlands', 'germany', 'italy', 'luxembourg']


In [125]:
nearest_neighbors('japan')

['japanese', 'korea', 'tokyo', 'china', 'thailand', 'indonesia', 'philippines', 'asia', 'vietnam', 'europe']


In [126]:
nearest_neighbors('einstein')

['relativity', 'freud', 'bohr', 'sigmund', 'postulated', 'theory', 'invented', 'analogy', 'maxwell', 'deduced']


In [127]:
nearest_neighbors('woman')

['girl', 'man', 'mother', 'boy', 'child', 'herself', 'victim', 'wife', 'she', 'teenager']


In [128]:
nearest_neighbors('nephew')

['cousin', 'grandson', 'brother', 'uncle', 'brother-in-law', 'niece', 'son-in-law', 'son', 'grandfather', 'eldest']


In [129]:
nearest_neighbors('february')

['october', 'december', 'september', 'january', 'august', 'november', 'june', 'april', 'july', 'march']
